L’évolution de la durée moyenne des films au fil des années.

In [48]:
# IMPORTS

import pandas as pd
import numpy as np

In [49]:
# BASES

df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_tmdb_full = pd.read_csv("BD/tmdb_full.csv")

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/581862119.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/581862119.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb_full = pd.read_csv("BD/tmdb_full.csv")


In [50]:
# Clean des tables

df_film = df_film[df_film['titleType'] == 'movie']
df_film['runtimeMinutes'].replace('\\N', np.nan, inplace = True)
df_film['runtimeMinutes'] = pd.to_numeric(df_film['runtimeMinutes'])
df_tmdb_full['runtime'].replace('\\N', np.nan, inplace = True)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/3789554848.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_film['runtimeMinutes'].replace('\\N', np.nan, inplace = True)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/3789554848.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [51]:
# MERGE
 
df_merge = pd.merge(df_film,
                    df_tmdb_full,
                    how = 'outer',
                    left_on = 'tconst',
                    right_on = 'imdb_id')

In [52]:
# Clean du Merge

df_merge = df_merge[~((df_merge['runtimeMinutes'].isna()) & (df_merge['runtime'].isna()))]
df_merge = df_merge[~((df_merge['runtimeMinutes'] == 0) & (df_merge['runtime'] == 0))]
df_merge = df_merge[~((df_merge['runtimeMinutes'] == 0) & (df_merge['runtime'].isna()))]
df_merge = df_merge[~((df_merge['runtimeMinutes'].isna()) & (df_merge['runtime'] == 0))]

# On transforme les types de données

df_merge['release_date'] = pd.to_datetime(df_merge['release_date'])
df_merge['release_date'] = df_merge['release_date'].replace('\\N', 0)
df_merge['release_date_year'] = df_merge['release_date'].dt.year
df_merge['startYear'] = df_merge['startYear'].replace('\\N', 0)
df_merge['startYear'] = pd.to_numeric(df_merge['startYear'], downcast="integer")

# On crée la colonne year_exact

df_merge['year_exact'] = df_merge[['release_date_year', 'startYear']].max(axis = 1)

# On supprime là où la donnée de yearexact est manquante

df_merge = df_merge[~(df_merge['year_exact'].isna())]
df_merge = df_merge[~(df_merge['year_exact'] == 0)]

In [53]:
# Calcul différence de runtime des 2 tables

df_merge['difference_runtime'] = abs(df_merge['runtime'] - df_merge['runtimeMinutes'])

In [54]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_merge[(df_merge['runtimeMinutes'] == df_merge['runtime'])]

# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/2474496667.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [55]:
df_a_travailler_1 = df_merge[~(df_merge['runtimeMinutes'] == df_merge['runtime'])]

In [56]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/3836658668.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [57]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [58]:
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/1682368460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [59]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/3265874609.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [60]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [61]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_7986/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [62]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [63]:
solution = df_final.groupby('year_exact')['runtime_exact'].mean()

In [64]:
solution

year_exact
1874.0      1.000000
1878.0      1.000000
1885.0      1.000000
1887.0      1.000000
1888.0      1.000000
             ...    
2024.0     93.420741
2025.0     92.701389
2026.0    104.593750
2027.0     72.000000
2029.0    112.500000
Name: runtime_exact, Length: 145, dtype: float64